In [1]:
from tensorflow import keras
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


import csv
import librosa
import librosa.display
import matplotlib.pyplot as plt
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

train_dir = os.path.abspath('../data/train/train')

classes = os.listdir(train_dir + '/audio')
classes.remove("_background_noise_")

with open(train_dir + '/validation_list.txt') as val_list:
    validation_list = [row[0] for row in csv.reader(val_list)]
assert len(validation_list) == 6798, "Validation files not loaded"

with open(train_dir + "/testing_list.txt") as test_list:
    testing_list = [row[0] for row in csv.reader(test_list)]
assert len(testing_list) == 6835, "testing files not loaded"

training_list  = []
all_files_list = []
class_counts = {}
train_audio_files = []
train_transcriptions=[]

for folder in classes:
    files = os.listdir(train_dir + '/audio/' + folder)
    for i, f in enumerate(files):
        all_files_list.append(folder+"/"+f)
        path = folder+'/'+f
        if path not in validation_list and path not in testing_list:
            training_list .append(folder+'/'+f)
            train_audio_files.append(train_dir+'/audio/'+path)
            train_transcriptions.append(folder)
        class_counts[folder] = i

#remove filenames from validation_list that don't exist anymore (due to eda)
validation_list = list(set(validation_list).intersection(all_files_list))
testing_list = list(set(testing_list).intersection(all_files_list))

assert len(validation_list) + len(testing_list) + len(training_list) == len(all_files_list), "Not All files splitted"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchaudio
from torchaudio.transforms import MelSpectrogram
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import ctc_loss

# Define your dataset class
class SpeechDataset(Dataset):
    def __init__(self, audio_files, transcriptions):
        self.audio_files = audio_files
        self.transcriptions = transcriptions
        self.transform = MelSpectrogram()

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio, sample_rate = torchaudio.load(self.audio_files[idx])
        audio = self.transform(audio)
        transcription = self.transcriptions[idx]
        return audio, transcription

# Define your Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_layers):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=4,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_dim,
        )
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, src, tgt):
        src = src.permute(2, 0, 1)  # Transformer expects shape (seq_len, batch, features)
        tgt = tgt.permute(1, 0)      # Transformer expects shape (seq_len, batch)
        out = self.transformer(src, tgt)
        out = self.linear(out)
        return out

# Define CTC loss
def ctc_loss_func(output, target, input_lengths, target_lengths):
    return ctc_loss(output, target, input_lengths, target_lengths)

# Other hyperparameters
input_dim = 128  # Assuming Mel spectrogram features
output_dim = len(set(train_transcriptions))  # Size of your vocabulary
hidden_dim = 256
num_layers = 4
batch_size=32
learning_rate=0.001

# Initialize your dataset and dataloader
train_dataset = SpeechDataset(train_audio_files, train_transcriptions)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize your model, optimizer, and other components
model = TransformerModel(input_dim, output_dim, hidden_dim, num_layers)
params = list(model.parameters())
optimizer = optim.Adam(params, lr=learning_rate)
# list(model.parameters())

# # Training loop
# for epoch in range(num_epochs):
#     for batch_idx, (audio, transcription) in enumerate(train_loader):
#         optimizer.zero_grad()
#         output = model(audio, transcription)
#         loss = ctc_loss_func(output, transcription, ...)
#         loss.backward()
#         optimizer.step()
#         print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}")

OSError: [WinError 182] The operating system cannot run %1. Error loading "c:\Users\megav\anaconda3\envs\iml-10\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.